In [45]:
import duckdb
# create a connection to a file called 'file.db'

def sql(sql):
    try:
        con = duckdb.connect('bpms.db')
        try:
            return con.sql(sql).df().copy()
        except Exception as e:
            return con.sql(sql)
    except Exception as e:
        raise(e)
    finally:
        con.close()

In [46]:
sql("update users set is_active=1")


In [40]:
sql('select * from users')

,user_id,username,email,phone,is_active
0,1,GENERAL_KHAN,general_khan@china.com,+860123456789,NaN
1,2,LT_GENERAL_PAWN,lt_general_pawn@china.com,+860987654321,NaN
2,3,POTUS,potus@china.com,+860111111111,NaN
3,4,ADMIN_JONES,admin_jones@china.com,+860222222222,NaN
4,5,ANALYST_SMITH,analyst_smith@china.com,+860333333333,NaN
5,6,REVIEWER_LEE,reviewer_lee@china.com,+860444444444,NaN
6,7,IT_JACKSON,it_jackson@china.com,+860555555555,NaN
7,8,MANAGER_WONG,manager_wong@china.com,+860666666666,NaN


In [6]:
# create a table and load data into it
sql("""
CREATE SEQUENCE IF NOT EXISTS seq_userid START 1;

CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_userid'),
    username VARCHAR NOT NULL UNIQUE,
    email VARCHAR NOT NULL UNIQUE,
    phone VARCHAR,
    is_active BOOLEAN NOT NULL DEFAULT 1
);
""")

In [7]:
# create a table and load data into it
sql("""
CREATE SEQUENCE IF NOT EXISTS seq_roleid START 1;

CREATE TABLE IF NOT EXISTS roles (
    role_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_roleid'),
    role_name VARCHAR NOT NULL UNIQUE,
    description TEXT
);

""")

In [8]:
# create a table and load data into it
sql("""
CREATE SEQUENCE IF NOT EXISTS seq_userroleid START 1;

-- User-Role mapping table
CREATE TABLE IF NOT EXISTS user_roles (
    user_role_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_userroleid'),
    user_id INTEGER NOT NULL,
    role_id INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (role_id) REFERENCES roles(role_id),
    UNIQUE (user_id, role_id) -- To ensure a user-role combination is unique
);

""")

In [9]:
# create a table and load data into it
sql("""
CREATE SEQUENCE IF NOT EXISTS seq_permissionid START 1;

-- Permissions table
CREATE TABLE IF NOT EXISTS permissions (
    permission_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_permissionid'),
    permission_name VARCHAR NOT NULL UNIQUE,
    description TEXT
);

""")

In [10]:
# create a table and load data into it
sql("""
CREATE SEQUENCE IF NOT EXISTS seq_rolepermissionid START 1;


-- Role-Permission mapping table
CREATE TABLE role_permissions (
    role_permission_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_rolepermissionid'),
    role_id INTEGER NOT NULL,
    permission_id INTEGER NOT NULL,
    FOREIGN KEY (role_id) REFERENCES roles(role_id),
    FOREIGN KEY (permission_id) REFERENCES permissions(permission_id),
    UNIQUE (role_id, permission_id) -- To ensure a role-permission combination is unique
);
""")

In [12]:
sql("""
        
CREATE SEQUENCE IF NOT EXISTS seq_auditid START 1;
        
CREATE TABLE IF NOT EXISTS bpms_audit_log (
    audit_id INTEGER PRIMARY KEY DEFAULT NEXTVAL('seq_auditid'),
    process_instance_id INTEGER NOT NULL,
    task_id INTEGER,
    timestamp TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    event_type VARCHAR,
    
    user_id INTEGER,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    
    role_id INTEGER,
    FOREIGN KEY (role_id) REFERENCES roles(role_id),
    
    permission_id INTEGER,
    FOREIGN KEY (permission_id) REFERENCES permissions(permission_id),
    
    action VARCHAR,
    old_value TEXT,
    new_value TEXT,
    comments TEXT,
    ip_address VARCHAR,
    status VARCHAR,
    error_details TEXT,
    associated_document_id INTEGER,
    outcome VARCHAR,
    duration INTEGER,
    external_system_reference VARCHAR,
    parent_process_id INTEGER,
    reason_for_change TEXT,
    data_payload TEXT
);
""")

In [13]:
sql("""

-- Inserting into users table
INSERT INTO users (username, email, phone) VALUES 
('GENERAL_KHAN', 'general_khan@china.com', '+860123456789'),
('LT_GENERAL_PAWN', 'lt_general_pawn@china.com', '+860987654321'),
('POTUS', 'potus@china.com', '+860111111111'),
('ADMIN_JONES', 'admin_jones@china.com', '+860222222222'),
('ANALYST_SMITH', 'analyst_smith@china.com', '+860333333333'),
('REVIEWER_LEE', 'reviewer_lee@china.com', '+860444444444'),
('IT_JACKSON', 'it_jackson@china.com', '+860555555555'),
('MANAGER_WONG', 'manager_wong@china.com', '+860666666666');

-- Inserting into roles table
INSERT INTO roles (role_name) VALUES 
('GENERAL_USER'),
('LT_GENERAL_USER'),
('PRESIDENT_USER'),
('ADMIN'),
('BUSINESS_ANALYST'),
('REVIEWER'),
('IT_SUPPORT'),
('MANAGER');

-- Inserting into permissions table
INSERT INTO permissions (permission_name) VALUES 
('view_tasks'),
('complete_tasks'),
('reassign_tasks'),
('override_decisions'),
('full_system_access'),
('access_analytics'),
('view_dashboards'),
('generate_reports'),
('review_processes'),
('access_audit_logs'),
('system_configuration'),
('handle_technical_issues'),
('integration_access'),
('view_team_tasks'),
('view_team_reports');

-- User-Role mapping
-- GENERAL_KHAN to GENERAL_USER role
INSERT INTO user_roles (user_id, role_id) 
SELECT 
    (SELECT user_id FROM users WHERE username = 'GENERAL_KHAN'),
    (SELECT role_id FROM roles WHERE role_name = 'GENERAL_USER');

-- LT_GENERAL_PAWN to LT_GENERAL_USER role
INSERT INTO user_roles (user_id, role_id) 
SELECT 
    (SELECT user_id FROM users WHERE username = 'LT_GENERAL_PAWN'),
    (SELECT role_id FROM roles WHERE role_name = 'LT_GENERAL_USER');

-- POTUS to PRESIDENT_USER role
INSERT INTO user_roles (user_id, role_id) 
SELECT 
    (SELECT user_id FROM users WHERE username = 'POTUS'),
    (SELECT role_id FROM roles WHERE role_name = 'PRESIDENT_USER');

-- ... (Continue similar mapping for other users)

-- Role-Permission mapping
-- GENERAL_USER to its permissions
INSERT INTO role_permissions (role_id, permission_id) 
VALUES 
    ((SELECT role_id FROM roles WHERE role_name = 'GENERAL_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'view_tasks')),
    ((SELECT role_id FROM roles WHERE role_name = 'GENERAL_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'complete_tasks'));

-- LT_GENERAL_USER to its permissions
INSERT INTO role_permissions (role_id, permission_id) 
VALUES 
    ((SELECT role_id FROM roles WHERE role_name = 'LT_GENERAL_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'view_tasks')),
    ((SELECT role_id FROM roles WHERE role_name = 'LT_GENERAL_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'complete_tasks')),
    ((SELECT role_id FROM roles WHERE role_name = 'LT_GENERAL_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'reassign_tasks'));

-- PRESIDENT_USER to its permissions
INSERT INTO role_permissions (role_id, permission_id) 
VALUES 
    ((SELECT role_id FROM roles WHERE role_name = 'PRESIDENT_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'view_tasks')),
    ((SELECT role_id FROM roles WHERE role_name = 'PRESIDENT_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'complete_tasks')),
    ((SELECT role_id FROM roles WHERE role_name = 'PRESIDENT_USER'), (SELECT permission_id FROM permissions WHERE permission_name = 'override_decisions'));

-- ... (Continue similar mapping for other roles and their permissions)
""")


In [20]:
sql('select * from information_schema.tables')

In [22]:
sql('select * from users')

IOException: IO Error: Could not set lock on file "/Users/mz/development/approv/bpms.db": Resource temporarily unavailable

In [8]:
sql('alter table users add column is_active boolean')